In [1]:
import warnings
import logging
import os

In [2]:
warnings.simplefilter('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

In [3]:
from  wandb_config import sweep_config
import wandb
import tensorflow as tf
from model import Transformer
from tensorflow.keras.optimizers import Adam
# from tensorflow_addons.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, Callback
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Add, Dense, LayerNormalization, Normalization , Masking, GlobalAveragePooling1D, Conv1D, Dropout, MultiHeadAttention, Layer
from tensorflow.keras.callbacks import LearningRateScheduler, Callback, ModelCheckpoint
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.metrics import Recall, Precision , AUC
from tensorflow_addons.metrics import F1Score
from tensorflow.keras import Model
import numpy as np




## Lr Scheduler

In [4]:

import math
# from tensorflow.keras.callbacks import Callback

# class PrintLR(Callback):
#     def on_epoch_end(self, epoch, logs = None):


def cosine_schedule(base_lr, total_steps, warmup_steps ):
    def step_fn(epoch):
        lr = base_lr
        epoch = 1

        progress = (epoch - warmup_steps) / float(total_steps -  warmup_steps)

        progress = tf.clip_by_value(progress, 0.0, 1.0)

        lr = lr * 0.5 * (1.0 + tf.cos(math.pi * progress))
        
        if warmup_steps:
            lr = lr * tf.minimum(1.0 , epoch/warmup_steps)
        
        return lr
    

    return step_fn

## Positional Embedding

In [5]:

class PositionalEmbedding(Layer):
    def __init__(self, units,dropout_rate,  **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)

        self.units = units
        self.conv_1 = Conv1D(filters  = units, kernel_size = 1)
        self.projection = Dense(units, kernel_initializer=TruncatedNormal(stddev=0.02))

        self.dropout = Dropout(rate=dropout_rate)

    def build(self, input_shape):
        super(PositionalEmbedding, self).build(input_shape)

        self.position = self.add_weight(
            name="position",
            shape=(1, input_shape[1], self.units),
            initializer=TruncatedNormal(stddev=0.02),
            trainable=True,
        )

    def call(self, inputs, training):
        x = self.projection(inputs)
        # x = self.conv_1(inputs)
        x = x + self.position
        return self.dropout(x, training=training)

## Encoder

In [6]:
class Encoder(Layer):
    def __init__(
        self, embed_dim, mlp_dim, num_heads, dropout_rate,
        attention_dropout_rate, **kwargs
    ):
        super(Encoder, self).__init__(**kwargs)
        #embed_dim = 128, 
        #mlp_dim = 256
        self.mha = MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_dim,
            dropout = attention_dropout_rate, 
            kernel_initializer = TruncatedNormal(stddev = 0.02)
        )

        self. dense_0 = Dense(
            units = mlp_dim, 
            activation = "gelu", 
            kernel_initializer = TruncatedNormal(stddev = 0.02)
        )

        self.dense_1 = Dense(
            units = embed_dim, 
            kernel_initializer = TruncatedNormal(stddev = 0.02)
        )

        self.conv_0 = Conv1D(filters = 4 , kernel_size = 1, activation = 'relu')
        self.conv_1 = Conv1D(filters  = embed_dim, kernel_size = 1)

        self.dropout_0 = Dropout(rate = dropout_rate)
        self.dropout_1 = Dropout(rate = dropout_rate)

        self.norm_0 = LayerNormalization(epsilon = 1e-6)
        self.norm_1 = LayerNormalization(epsilon = 1e-6)

        self.add_0 = Add()
        self.add_1 = Add()
    
    def call(self, inputs, training , mask):


        x = self.norm_0(inputs)
        x = self.mha(
            query = x, 
            value = x, 
            key = x,
            attention_mask = mask,
            training = training
        ) #[batch_size, sequence_length, embed_dim][8, 500, 3]
        x = self.dropout_0(x, training= training)
        x = self.add_0([x, inputs])

        #MLP block 
        y = self.norm_1(x)
        y = self.dense_0(y) #[batch_size , sequence_length, neurons]
        y = self.dropout_1(y, training)
        y = self.dense_1(y)#[batch_size , sequence_lenght, neurons]
        y = self.dropout_1(y, training)
        

        return self.add_1([x, y])

## Transformer

In [7]:
class Transformer(Model):
    def __init__(
        self,
        num_layers,
        embed_dim,
        mlp_dim,
        num_heads,
        num_classes,
        dropout_rate,
        attention_dropout_rate,
        **kwargs
    ):
        super(Transformer, self).__init__(**kwargs)

        # Input (normalization of RAW measurements)
        self.input_norm = Normalization()
        
        #Making Layer
        self.masking_layer = Masking(mask_value = 0.0)

        # Input
        self.pos_embs = PositionalEmbedding(embed_dim, dropout_rate)

        # Encoder
        self.e_layers = [
            Encoder(embed_dim, mlp_dim, num_heads, dropout_rate, attention_dropout_rate)
            for _ in range(num_layers)
        ]

        # Output
        self.norm = LayerNormalization(epsilon=1e-5)
        self.pool = GlobalAveragePooling1D(data_format = 'channels_first')
        self.dense_0 = Dense(mlp_dim, activation = 'relu')
        self.final_layer = Dense(1, kernel_initializer="zeros", activation = 'sigmoid')

    def call(self, inputs, training = True):
        expanded_input = tf.cast(tf.tile(tf.expand_dims(inputs, axis=-2), [1, 1, 500,1]), tf.float32)
        self.masking_layer.build(expanded_input.shape)
        mask = self.masking_layer.compute_mask(expanded_input)
        x = self.input_norm(inputs) 
        x = self.pos_embs(x, training=training)
        for layer in self.e_layers:
            x = layer(x, training=training , mask = mask)
        x = self.norm(x)
        x = self.pool(x)
        x = self.dense_0(x)

        x = self.final_layer(x)
        return x

## Train Function 

In [8]:
def train():
    config = {
      'epochs': 50,
      'num_layers':  3,
      'embed_layer_size': 3,
      'global_clipnorm' : 3.0,
      'fc_layer_size': 256,
      'num_heads': 2,
      'dropout': 0.1,
      'attention_dropout': 0.1,
      'optimizer': 'adam',
      'amsgrad': False,
      'label_smoothing': 0.1,
      'learning_rate': 1e-3,
      #'weight_decay': {
      #    'values': [2.5e-4, 1e-4, 5e-5, 1e-5]
      'warmup_steps': 5,
      'batch_size': 8}
  

    # config = wandb.config
    
    # Generate new model
    model = Transformer(
      num_layers=config['num_layers'],
      embed_dim=config['embed_layer_size']
      ,
      mlp_dim=config['fc_layer_size'],
      num_heads=config['num_heads'],
      num_classes=2,
      dropout_rate=config['dropout'],
      attention_dropout_rate=config['attention_dropout'],
    )

    

    # adapt on training dataset - must be before model.compile !!!
    model.input_norm.adapt(X_train, batch_size=config['batch_size'])
    # print(model.input_norm.variables)

    # Select optimizer
    if config['optimizer'] == "adam":
      optim = Adam(
          global_clipnorm=config['global_clipnorm'],
          amsgrad=config['amsgrad'],
      )
    # elif config.optimizer == "adamw":
    #   optim = AdamW(
    #       weight_decay=config.weight_decay,
    #       amsgrad=config.amsgrad,
    #       global_clipnorm=config.global_clipnorm,
    #       exclude_from_weight_decay=["position"]
    #   )
    else:
      raise ValueError("The used optimizer is not in list of available")

    model.compile(
      loss= BinaryCrossentropy(label_smoothing=config['label_smoothing']),
      optimizer=optim,
      metrics=[Recall(), Precision() , AUC(), F1Score(num_classes = 1)],
    )
    
    checkpoint_filepath = os.path.join(os.getcwd(), 'tmp/chekcpoint')
    model_checkpoint = ModelCheckpoint(filepath = checkpoint_filepath, 
                                      save_weights_only = True, 
                                      monitor = 'val_precision', 
                                      mode = 'max', 
                                      save_best_only = True, 
                                      verbose = True)


    # Train model
    history = model.fit(
      X_train,
      y_train,
      batch_size=config['batch_size'],
      epochs=config['epochs'],
      validation_data=(X_val, y_val),
      callbacks=[
        LearningRateScheduler(cosine_schedule(base_lr=config['learning_rate'], total_steps=config['epochs'], warmup_steps=config['warmup_steps'])),
        EarlyStopping(monitor="val_loss", mode='min', min_delta=0.001, patience=5),
        model_checkpoint
      ],
      verbose=1
    )
    return model

In [ ]:
    model = None
    dataset_path = os.path.join(os.getcwd(), 'fall_detection_dataset.npz')
    f = np.load(dataset_path)
    signals = f['trials']

    labels = f['labels']

    # split to train-test
    X_train, X_test, y_train, y_test = train_test_split(
        signals, labels, test_size=0.15, random_state=9, stratify=labels
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.15, random_state=9, stratify=y_train
    )

    # sweep_id = wandb.sweep(sweep_config, project="KD_Transformer")
    with tf.device('/gpu:0'):
      model = train()

Metal device set to: Apple M1 Pro
Epoch 1/50
82/82 [==============================] - ETA: 0s - loss: 0.6927 - recall: 0.9003 - precision: 0.5581 - auc: 0.5191 - f1_score: 0.6741

In [32]:
score = model.evaluate(X_test, y_test, batch_size=8 , steps = X_test.shape[0] / 8)

17/17 [==============================] - 1s 71ms/step - loss: 0.4293 - recall_6: 0.8841 - precision_6: 0.8133 - auc_6: 0.9266


In [34]:
f1_score = 2/((1/score[1]) + (1/score[2]))

In [35]:
f1_score

0.8472222344846358

In [17]:
os.path.join(os.getcwd(), 'tmp/chekcpoint')

'/Users/tousif/Lstm_transformer/KD_Multimodal/tmp/chekcpoint'